<a href="https://colab.research.google.com/github/kieranlim-glitch/futurescalculator/blob/master/LP_movement_filtered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import re

INPUT_FILE = "/content/transactions_report_1770443011361.csv"

CODE_TO_LP = {
    "AN": "Asianext",
    "B2": "B2C2",
    "BN": "Binance",
    "CM": "Cumberland",
    "DBS": "DBS",
    "DEB": "Deribit",
    "LM": "LMAX",
    "CIR": "Trading Circle",
    "KGI": "KGI",
    "OKX": "OKX",
    "FID": "Fidelity",
    "TRI": "Trillionx",
    "WM": "Wintermute",
    "ZOD": "Zodia",
    "GX": "Galaxy",
    "PX": "PX",
}

CODE_REGEX = re.compile(
    r"^\s*(" + "|".join(CODE_TO_LP.keys()) + r")\s*:",
    re.IGNORECASE
)

def detect_lp(note):
    if pd.isna(note):
        return None, None
    match = CODE_REGEX.search(str(note))
    if not match:
        return None, None
    code = match.group(1).upper()
    return code, CODE_TO_LP[code]

df = pd.read_csv(INPUT_FILE)

df["LP_Code"], df["LP_Name"] = zip(*df["Note"].apply(detect_lp))

# remove duplicate CTA -> Corporate funding
CORP_KEY = "HKT - Corporate"
CTA_BLOCK = r"(Customer Trust|Trust Account|CTA)"

is_cta_to_corp = (
    df["Source"].astype(str).str.contains(CTA_BLOCK, case=False, na=False) &
    df["Destination"].astype(str).str.contains(CORP_KEY, case=False, na=False)
)

df_lp = df[
    (df["Status"] == "COMPLETED") &
    (df["LP_Code"].notna()) &
    (~is_cta_to_corp)
].copy()

# --- Direction ---
# Anything arriving into these is considered "Receive"
# B2C2 still sends to CTA
RECEIVE_KEYS = [
    "HKT - Corporate",
    "HKT - Customer Trust Account - Warm",
    "Hako Tech Main",
]

# Single regex like "A|B|C"
receive_regex = "|".join(map(re.escape, RECEIVE_KEYS))

df_lp["Direction"] = None

# Destination matches any receive key
df_lp.loc[
    df_lp["Destination"].astype(str).str.contains(receive_regex, case=False, na=False),
    "Direction"
] = "Receive"

# Source is Corporate (Original Rule)
df_lp.loc[
    df_lp["Source"].astype(str).str.contains(CORP_KEY, case=False, na=False),
    "Direction"
] = "Send to LP"

# Check for blanks
df_lp["Direction"] = df_lp["Direction"].fillna("Other")

df_lp = df_lp[
    [
        "Date",
        "LP_Code",
        "LP_Name",
        "Direction",
        "Asset Symbol",
        "Amount",
        "USD Amount",
        "Source",
        "Destination",
        "TxHash",
        "Note",
    ]
].sort_values("Date")

OUTPUT_FILE = "/content/lp_settlements_filtered_final1.csv"
df_lp.to_csv(OUTPUT_FILE, index=False)

print("Saved:", OUTPUT_FILE)
print("Rows kept:", len(df_lp))
df_lp.head(10)


Saved: /content/lp_settlements_filtered_final1.csv
Rows kept: 570


/tmp/ipython-input-4043061232.py:48: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["Source"].astype(str).str.contains(CTA_BLOCK, case=False, na=False) &


,Date,LP_Code,LP_Name,Direction,Asset Symbol,Amount,USD Amount,Source,Destination,TxHash,Note
4334,02 Feb 2026 01:21:00 GMT,BN,Binance,Send to LP,USDT,5000000.00,4.995910e+06,HKT - Corporate - Warm,Hako Tech Main,b25ab77c0beadefe41172b5eb5d156f9850836e7609f3b...,BN: https://coinhako.slack.com/archives/C04EG8...
4319,02 Feb 2026 01:32:40 GMT,BN,Binance,Receive,USDC,5000000.00,4.998269e+06,Hako Tech Main,HKT - Corporate - Warm,0x098201460b752609cbeb0307fefab7ff347b5741b3be...,BN: https://coinhako.slack.com/archives/C04EG8...
4290,02 Feb 2026 02:15:28 GMT,DEB,Deribit,Receive,ETH,500.00,1.139913e+06,External,HKT - Corporate - Warm,0xc0df435d4a500d7c0330f14dda068c3534351d3cf285...,DEB: https://coinhako.slack.com/archives/C088U...
4269,02 Feb 2026 02:29:36 GMT,LM,LMAX,Send to LP,SOL,14947.97,1.524724e+06,HKT - Corporate - Warm 2,LMAX Digital HT,5rESLjkHtV36uEY7rcuFtUu2ot5ABCBKrUzPZUMy1kSCiH...,LM: https://coinhako.slack.com/archives/C04EXR...
4217,02 Feb 2026 02:42:56 GMT,DEB,Deribit,Receive,ETH,1440.00,3.274596e+06,External,HKT - Corporate - Warm,0xab270c0a5f53e49f710425fd59ae29e748f39230f8dd...,DEB: https://coinhako.slack.com/archives/C088U...
4184,02 Feb 2026 02:51:06 GMT,B2,B2C2,Send to LP,BTC,50.97,3.922794e+06,HKT - Corporate - Warm,B2C2 yusho,35fc66bfce62060c5468c60472b4c33ad66e3370ea3021...,B2: https://coinhako.slack.com/archives/G01NQG...
4182,02 Feb 2026 02:51:21 GMT,B2,B2C2,Send to LP,SOL,1106.50,1.119478e+05,HKT - Corporate - Warm 2,B2C2 yusho,K1RF1mrdt6YyPQjh1YYnb6VGMhdPFfZm5817wiXuJhJDRu...,B2: https://coinhako.slack.com/archives/G01NQG...
4168,02 Feb 2026 02:56:35 GMT,B2,B2C2,Send to LP,AVAX,5812.74,5.790372e+04,HKT - Corporate - Warm,B2C2 yusho,0x95134a42f6794eb7a27f359e6e0fa04157d544f3445c...,B2: https://coinhako.slack.com/archives/G01NQG...
4139,02 Feb 2026 03:36:25 GMT,B2,B2C2,Send to LP,BCH,341.36,1.711574e+05,HKT - Corporate - Warm,B2C2 yusho,1a66295f256235c36f909997bd3d21aa0bbe548d6197b9...,B2: https://coinhako.slack.com/archives/G01NQG...
4124,02 Feb 2026 03:57:08 GMT,B2,B2C2,Receive,XRP,98100.00,1.537680e+05,B2C2 yusho,HKT - Customer Trust Account - Warm,0E9A9E2B6867598B8E0C08E932E640616956AEE95EB607...,B2: https://coinhako.slack.com/archives/G01NQG...
